In [49]:
import requests
import pandas as pd
import random
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import re

In [50]:
MAIN_API = 'https://fantasy.premierleague.com/drf/bootstrap-static'
TEAMS_API = 'https://fantasy.premierleague.com/drf/teams'
TYPES_API = 'https://fantasy.premierleague.com/drf/element-types/'
PLAYER_API = 'https://fantasy.premierleague.com/drf/element-summary/'
MY_TEAM_API = 'https://fantasy.premierleague.com/drf/my-team/2083687'
MY_TRANSFERS_API = 'https://fantasy.premierleague.com/drf/transfers'
MY_LEAGUES = 'https://fantasy.premierleague.com/drf/leagues-entered/2083687'
MY_LEAGUE_STANDING = 'https://fantasy.premierleague.com/drf/leagues-classic/'
ENTRIES = 'https://fantasy.premierleague.com/drf/entries'
EVENTS = 'https://fantasy.premierleague.com/drf/events'

In [51]:
def get_df(api, **kwargs):
    data = requests.get(api).json()
    #print(data.keys())
    for key, value in kwargs.items():
        if key =='arg':
            data = data[value]
    return pd.DataFrame.from_dict(data, orient = 'columns')

In [52]:
def login(username, password, api):
    # Fill in your details here to be posted to the login form.
    payload = {
        'login': username,
        'password': password,
        'app' :'plfpl-web',
        'redirect_uri': 'https://fantasy.premierleague.com/a/login'
    }

    # Use 'with' to ensure the session context is closed after use.
    with requests.Session() as s:

        s.get('https://fantasy.premierleague.com')
        csrftoken = s.cookies["csrftoken"]
        payload['csrfmiddlewaretoken'] = csrftoken
        p = s.post('https://users.premierleague.com/accounts/login/', data=payload)
        #print(p.text)

        #print(p.text)
        # An authorised request.
        r = s.get(api)
        
    return r.json()

In [53]:
def get_team_info(data, team, **kwargs):
    teams = get_df(TEAMS_API)
    teams = teams[['id', 'name']]
    team_names = teams.set_index('id').to_dict()['name']
    types = get_df(TYPES_API)
    types = types[['id', 'singular_name_short']]
    types = types.set_index('id').to_dict()['singular_name_short']
    n_df = df.replace({'team': team_names, 'element_type': types})
    return n_df[n_df['team'] == team]

In [6]:
 def get_player_info(df, name, tag):
    player_ids = df[['id', 'web_name']]
    i = player_ids[player_ids['web_name'] == name]['id'].values[0]
    api = PLAYER_API + str(i)
    x = get_df(api, arg = tag)
    return x

## Main API

In [54]:
df = get_df(MAIN_API, arg='elements')
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,3,130,100.0,100.0,1,11334,0,0,-1,...,0.0,24,79860,12,121560,51,0.0,4.9,Cech,0
1,0,2,176,NaN,NaN,1,80201,0,0,-2,...,0.0,31,92716,177,97637,209,0.6,6.5,Leno,0
2,0,0,11,100.0,100.0,0,51507,0,0,-1,...,7.0,0,6355,91,5475,31,0.0,0.0,Koscielny,1
3,4,3,261,0.0,0.0,3,98745,-1,1,-1,...,274.0,48,557389,22,860125,8118,0.3,8.7,Bellerín,3
4,2,3,156,100.0,100.0,2,38411,0,0,-1,...,163.0,36,207550,440,178853,154,0.2,6.7,Monreal,1


In [55]:
df.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'news_added', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'red_cards', 'saves', 'second_name', 'selected_by_percent', 'special',
       'squad_number', 'status', 'team', 'team_code', 'threat', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'yellow_cards'],
      dtype='object')

## Club Information

In [56]:
cols = ['id', 'web_name', 'total_points', 'points_per_game', 'team' , 'element_type', 'selected_by_percent']
get_team_info(df, 'Man City')[cols].head()

## Player Specific Info

In [58]:
get_player_info(df, 'Fellaini','history').head()

,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards
0,0,0,0,0,0,4,0,2,0,0.0,...,2.0,0.0,1,0,0,0,50,True,0,0
1,1,16,0,0,0,12,0,0,12,3.7,...,3.0,2.0,4,-1773,2101,3874,50,False,0,0
2,0,21,0,0,0,5,0,0,15,17.3,...,0.0,4.0,1,-248,4162,4410,50,True,0,0
3,0,54,0,0,0,16,1,7,50,4.8,...,0.0,0.0,3,-2770,2185,4955,50,False,0,0
4,1,65,0,0,0,24,0,6,53,12.4,...,1.0,4.0,5,-2573,3694,6267,50,False,0,0


## Personal Team and Misc Information

### Team Information

In [ ]:
data = login(<email_id>, <password, MY_TRANSFERS_API)

In [60]:
data = login('ranjan31051997@gmail.com', 'Ranjan#105', MY_TRANSFERS_API)
print(data.keys())

In [63]:
x = pd.DataFrame(data['picks'])
x.head()

,element,entry,id,is_captain,is_vice_captain,multiplier,position,purchase_date,purchase_price,selling_price
0,400,2083687,31255291,False,False,1,1,2018-12-21,47,47
1,155,2083687,31255292,False,False,1,12,2018-08-03,40,40
2,331,2083687,31255293,False,False,1,4,2018-12-05,41,41
3,145,2083687,31255294,False,False,1,14,2018-08-03,40,41
4,409,2083687,31255295,False,False,1,3,2018-12-04,43,43


In [28]:
l = df[['id', 'web_name']]
names = l.set_index('id').to_dict()['web_name']
names_df = x.replace({'element': names})

In [32]:
names_df.head()

### Details of a particular league

In [20]:
#Only if you created the league
i = 657907
data = login('ranjan31051997@gmail.com', 'Ranjan#105', MY_LEAGUE_STANDING + str(i))

In [21]:
data.keys()

dict_keys(['league', 'entries'])

In [22]:
x = pd.DataFrame(data['entries'])
x

,entry,entry_name,id,joined_time,league,player_first_name,player_last_name
0,2083687,BootyGuy,16765657,2018-08-07T12:25:55Z,657907,Ranjan,Srinivas
1,3096386,Anything,18227032,2018-08-08T09:23:37Z,657907,Sreevatsan,Murthy
2,2895680,God Save the Queen,24387457,2018-08-10T08:10:14Z,657907,Kavialagan,Arjunan
3,4214519,Star Ash,26170484,2018-08-10T13:05:34Z,657907,Jebin,Moses


### Events

In [45]:
data = login('ranjan31051997@gmail.com', 'Ranjan#105', EVENTS)
x = pd.DataFrame(data)
x.head()